In [3]:
from ultralytics import YOLO
import cv2
from IPython.display import display, Image
import re

In [14]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

from paddleocr import PaddleOCR

##### Import image to test

In [4]:
img = 'C:/Users/EliottCOLSON/OneDrive - Ekimetrics/Documents/carbondiet/data/images/menu2.png'
image = cv2.imread(img)

### Run box detection model

In [5]:
model = YOLO('../data/weights/best_v5.pt')

In [6]:
results = model(source=img)


image 1/1 C:\Users\EliottCOLSON\OneDrive - Ekimetrics\Documents\carbondiet\data\images\menu2.PNG: 800x480 9 Recipes, 1 Section title, 1 Section, 971.7ms
Speed: 3.0ms preprocess, 971.7ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)


In [ ]:
res_plotted = results[0].plot()
cv2.imshow("result", res_plotted)
cv2.waitKey(0)

### Adjust box detection & section 

### Extract text from boxes

In [7]:
recipes_index = [i for i in range(len(results[0].boxes.cls)) if int(results[0].boxes.cls[i].item())==0]

In [19]:
list_text_tesseract = []
for i in recipes_index:
    box_coord = results[0].boxes.xywh[i]
    x1 = (box_coord[0].item())
    y1 = (box_coord[1].item())
    w = int(box_coord[2].item())
    h = int(box_coord[3].item())
    x = int(x1 - (w / 2))
    y = int(y1 - (h / 2))
    ROI = image[y:y+h,x:x+w]
    text = pytesseract.image_to_string(ROI, config='--psm 6')
    list_text_tesseract.append(text.lower())

In [20]:
list_text_tesseract

['gaspacho et sa quenelle au chévre frais 9,50 €\n\x0c',
 'saumon mariné facon gravlax et sa sauce 13,00 €\n\x0c',
 'carpaccio de tomates anciennes et buratina 12,00 €\n\x0c',
 'rillettes de lapin a la sarriette 10,00 €\n\x0c',
 'escargots de bourgogne, beurre fin a pail\npar6 850€  parl2 16,00€\n\x0c',
 'salade de lentilles vertes du berry 8,50€\nchips de lard\n\x0c',
 'soupe a ?oignon gratinée 9,00 €\n\x0c',
 'salade de lentilles vertes du berry 8,50€\nchips de lard\n\x0c',
 'terrine de foie gras de canard, 19,00 €\nchutney du moment, et pain poilane\n\x0c']

In [15]:
pytesseract.image_to_string(image, config='--psm 6')

'A lie 09:06 Ca eed\nme\nHee ae ea\na rg a ef fa i\nLee fi yee , (\ni yy la fe i :\nMCAS Stas eesti\n——_\n> wa —s ha\nEntrées\nSalade de Lentilles Vertes du Berry 850€\nChips de Lard\nSoupe a ?Oignon Gratinée 9,00 €\nGaspacho et sa Quenelle au Chévre Frais 9,50 €\nRillettes de Lapin a la Sarriette 10,00 € I\nCarpaccio de Tomates Anciennes et Buratina 12,00 €\nSaumon Mariné Facon Gravlax et sa Sauce 13,00 € i\nEscargots de Bourgogne, Beurre Fin a PAil\npar6 850€  parl2 16,00€\nTerrine de Foie Gras de Canard, 19,00 €\nChutney du Moment, et Pain Poilane\nUn tableau des allergénes présents dans nos plats,\nest disponible 4 Paccueil du restaurant\nvw\nx a we\n\x0c'

In [23]:
ocr = PaddleOCR(use_angle_cls=True, enable_mkldnn=True) # need to run only once to download and load model into memory

[2023/03/22 15:25:25] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\EliottCOLSON/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\EliottCOLSON/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pg

In [24]:
list_text_paddle = []
for i in recipes_index:
    box_coord = results[0].boxes.xywh[i]
    x1 = (box_coord[0].item())
    y1 = (box_coord[1].item())
    w = int(box_coord[2].item())
    h = int(box_coord[3].item())
    x = int(x1 - (w / 2))
    y = int(y1 - (h / 2))
    ROI = image[y:y+h,x:x+w]
    #text = pytesseract.image_to_string(ROI,lang="fra", config='--psm 6')
    #list_text.append(re.sub('[\n|\x0c]', '', text).lower())
    result = ocr.ocr(ROI, cls=True)
    list_text_paddle.append([x[1][0] for x in result[0]])

list_text_paddle

[2023/03/22 15:25:30] ppocr DEBUG: dt_boxes num : 2, elapse : 0.3583347797393799
[2023/03/22 15:25:31] ppocr DEBUG: cls num  : 2, elapse : 0.15430498123168945
[2023/03/22 15:25:31] ppocr DEBUG: rec_res num  : 2, elapse : 0.49771642684936523
[2023/03/22 15:25:31] ppocr DEBUG: dt_boxes num : 2, elapse : 0.16746234893798828
[2023/03/22 15:25:31] ppocr DEBUG: cls num  : 2, elapse : 0.030261754989624023
[2023/03/22 15:25:32] ppocr DEBUG: rec_res num  : 2, elapse : 0.23590731620788574
[2023/03/22 15:25:32] ppocr DEBUG: dt_boxes num : 2, elapse : 0.08517861366271973
[2023/03/22 15:25:32] ppocr DEBUG: cls num  : 2, elapse : 0.014993667602539062
[2023/03/22 15:25:32] ppocr DEBUG: rec_res num  : 2, elapse : 0.23744916915893555
[2023/03/22 15:25:32] ppocr DEBUG: dt_boxes num : 2, elapse : 0.029938220977783203
[2023/03/22 15:25:32] ppocr DEBUG: cls num  : 2, elapse : 0.017024993896484375
[2023/03/22 15:25:32] ppocr DEBUG: rec_res num  : 2, elapse : 0.21451282501220703
[2023/03/22 15:25:32] ppocr D

[['Gaspacho et sa Quenelle au Chevre Frais', '9,50'],
 ['Saumon Marine Facon Gravlax et sa Sauce', '13,00'],
 ['Carpaccio de Tomates Anciennes et Buratina', '12,00'],
 ['Rillettes de Lapin a la Sarriette', '10.00'],
 ['par 6 8,50 ', 'par 12 16,00'],
 ['Salade de Lentilles Vertes du Berry', '8,50', 'ChibrdeLard'],
 ['Soupe a POignon Gratinee', '9,00'],
 ['Salade de Lentilles Vertes du Berry', '8,50', 'ChipsdeLard'],
 ['Terrine de Foie Gras de Canard.',
  '19,00',
  'Chutney du Moment, et Pain Poilane']]

In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [11]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-printed')

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [13]:
list_text_trocr = []
for i in recipes_index:
    box_coord = results[0].boxes.xywh[i]
    x1 = (box_coord[0].item())
    y1 = (box_coord[1].item())
    w = int(box_coord[2].item())
    h = int(box_coord[3].item())
    x = int(x1 - (w / 2))
    y = int(y1 - (h / 2))
    ROI = image[y:y+h,x:x+w]    

    pixel_values = processor(images=ROI, return_tensors="pt").pixel_values

    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    list_text_trocr.append(generated_text)

list_text_trocr

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['GASPACHO ET SA QUENGLE AU CHEVIE PRAS 9,50 E',
 'SAUMON MARINE FACON GRASLAX ET SA SAUCE 13,00 6',
 'CAPPACIO DE TOMATES ANCHENES ET BURATINA 12,006',
 'RELETIES DE LAPIN A LA SARRITEITE',
 'EXCHANGES DE BOUNDSING, BATE 8,506 PAP 12 16,006',
 'SABLE DE TERMILES VERIES DU BERRY',
 'SOUPED POUGH ONLINE',
 'CAPILING LARVILLES, VEMES DU BERRY',
 'TEMING AS FACE, OFFING DETAILS ARE AN OFFER']

In [ ]:
def is_contained(outer_box, inner_box):
    x1, y1, w1, h1 = outer_box
    x2, y2, w2, h2 = inner_box

    if x2 >= x1 and y2 >= y1 and (x2 + w2) <= (x1 + w1) and (y2 + h2) <= (y1 + h1):
        return True
    else:
        return False

In [ ]:
img = 'C:/Users/EliottCOLSON/OneDrive - Ekimetrics/Documents/carbondiet/data/images/menu2.png'

In [ ]:
# Custom libraries
import sys
sys.path.append('./')

In [2]:
import cv2

In [1]:
from project.ComputerVision.box_detection import BoundingBoxDetection

In [3]:
boxdetect = BoundingBoxDetection()

In [ ]:
BoundingBoxDetection.detect_boxes(img)

In [2]:
import json

In [3]:
file = open("../data/other/box_example.json", 'r')
box_example = json.load(file)

In [11]:
test_list = [(x[0], x[1]) for x in box_example['Recipe']]

In [32]:
def remove_overlapping_bounding_boxes(bounding_boxes, treshold=0.5):
    #input a list of tuple containing bounding boxes and confidence
    #output a cleaned (according to fixed treshold) list of tuple containing bounding boxes and confidence
    cleaned_boxes = {}
    for box, prob in bounding_boxes:
        x1, y1, w1, h1 = box
        area1 = w1 * h1
        added = False
        for key in cleaned_boxes:
            x2, y2, w2, h2 = cleaned_boxes[key][0]
            area2 = w2 * h2
            x_overlap = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
            y_overlap = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
            overlap_area = x_overlap * y_overlap
            if overlap_area > treshold * area1 or overlap_area > treshold * area2:
                if prob > cleaned_boxes[key][1]:
                    cleaned_boxes[key] = (box, prob)
                added = True
                break
        if not added:
            cleaned_boxes[len(cleaned_boxes)] = (box, prob)
    return list(cleaned_boxes.values())

In [19]:
len(test_list)

36

In [20]:
len(remove_overlapping_bounding_boxes(test_list))

25

In [21]:
import cv2

In [41]:
img = 'C:/Users/EliottCOLSON/OneDrive - Ekimetrics/Documents/carbondiet/data/images/menu7.jpg' # copy absolute path toward image
image = cv2.imread(img)

In [31]:
image2 = image.copy()
list_box = [x[0] for x in remove_overlapping_bounding_boxes(test_list, 0.5)]

for box in list_box:
    x, y, w ,h = box
    cv2.rectangle(image2, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('Image with rectangle', image2)
cv2.waitKey(0)

-1

In [33]:
from ultralytics import YOLO

In [38]:
yolo_model = YOLO('C:/Users/EliottCOLSON/OneDrive - Ekimetrics/Documents/carbondiet/data/weights/best_v6.pt')

In [43]:
results = yolo_model(image)


0: 800x800 36 Recipes, 1 Section title, 844.9ms
Speed: 9.0ms preprocess, 844.9ms inference, 2.5ms postprocess per image at shape (1, 3, 800, 800)


In [1]:
import json

In [16]:
file = open("../data/other/box_example.json", 'r')
box_raw = json.load(file)

In [17]:
box_example = {"Recipe" : [x[0] for x in box_raw["Recipe"]],
               "Section title" : [x[0] for x in box_raw["Section title"]]}

In [37]:
import math

def attribute_boxes(dict_boxes, recipe_name = "Recipe", section_title_name = "Section title"):
    # Function that takes a dict of boxes with recipes and section itles. And then assigns each recipe to the closest section title box located above.

    recipe = dict_boxes[recipe_name]
    section = dict_boxes[section_title_name]
    
    # Initialize dictionary to hold assigned boxes
    assigned_boxes = {}

    # Initialize list to hold unassigned recipe boxes
    unassigned_boxes = []

    # Sort recipe boxes by y-coordinate
    recipe = sorted(recipe, key=lambda box: box[1])

    # Loop over recipe boxes
    for recipe_box in recipe:
        # Initialize variables to keep track of closest section box and distance
        closest_section_box = None
        closest_distance = float('inf')

        # Loop over section boxes
        for section_box in section:
            # Check if section box is above recipe box
            if section_box[1] > recipe_box[1]:
                continue
            
            # Compute distance between centers of recipe box and section box
            recipe_center = (recipe_box[0] + recipe_box[2]/2, recipe_box[1] + recipe_box[3]/2)
            section_center = (section_box[0] + section_box[2]/2, section_box[1] + section_box[3]/2)
            distance = math.sqrt((recipe_center[0] - section_center[0])**2 + (recipe_center[1] - section_center[1])**2)
            # Check if distance is smaller than current closest distance
            if distance < closest_distance:
                closest_section_box = section_box
                closest_distance = distance

        # Check if a section box was found for the recipe box
        if closest_section_box is not None:
            # Add recipe box to dictionary of assigned boxes
            if section.index(closest_section_box) in assigned_boxes:
                assigned_boxes[section.index(closest_section_box)].append(recipe_box)
            else:
                assigned_boxes[section.index(closest_section_box)] = [recipe_box]
        else:
            # Add recipe box to list of unassigned boxes
            unassigned_boxes.append(recipe_box)

    # Add unassigned boxes to dictionary of assigned boxes
    assigned_boxes['unassigned'] = unassigned_boxes

    return assigned_boxes


In [38]:
attribute_boxes(box_example)

{0: [[495, 86, 428, 82],
  [53, 91, 421, 56],
  [484, 136, 453, 31],
  [55, 148, 411, 49],
  [497, 168, 425, 58],
  [54, 203, 416, 52],
  [496, 238, 427, 71],
  [55, 258, 416, 44],
  [63, 273, 365, 32],
  [55, 308, 416, 70],
  [500, 316, 422, 73],
  [71, 324, 409, 52],
  [55, 379, 415, 76],
  [499, 394, 423, 62],
  [71, 400, 376, 54],
  [534, 406, 388, 39],
  [54, 456, 420, 66],
  [501, 462, 419, 49],
  [508, 495, 414, 74],
  [500, 525, 421, 46],
  [55, 531, 425, 47],
  [501, 575, 427, 93],
  [55, 582, 428, 65],
  [54, 583, 426, 43],
  [55, 603, 427, 104],
  [54, 641, 421, 67],
  [497, 666, 426, 65],
  [519, 684, 397, 38],
  [55, 707, 418, 50],
  [79, 723, 397, 38],
  [497, 739, 421, 59],
  [53, 764, 418, 49],
  [83, 781, 392, 31],
  [497, 806, 415, 57],
  [48, 816, 425, 96],
  [498, 869, 413, 56]],
 'unassigned': []}